## 라이브러리

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd
from tqdm import tqdm
import calendar
import datetime

## 데이터 수집

In [ ]:
def get_billboard_chart(date):
    url = f'https://www.billboard.com/charts/hot-100/{date}/'
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = pd.DataFrame(columns = ['Week', 'Rank', 'Title', 'Artist'])
    week = soup.select_one('#post-1479786 > div.pmc-paywall > div > div > div > div.lrv-u-flex.lrv-u-align-items-center.lrv-u-justify-content-space-between > p').text.strip()

    for song in soup.select('ul.o-chart-results-list-row'):
            rank = song.select_one('span.c-label').text.strip()
            title = song.select_one('h3.c-title').text.strip()
            artist = song.select_one('span.c-label.a-no-trucate').text.strip()
            song_info = pd.DataFrame({
                'Week': [week],
                'Rank': [rank],
                'Title': [title],
                'Artist': [artist]
            })
            songs = pd.concat([songs, song_info])

    return songs

date_list = pd.date_range(start = '20181230', end = '20231231', freq = 'W-SUN')
date_list = [str(d)[:10] for d in list(date_list)]

songs = pd.DataFrame()
for date in tqdm(date_list):
    weekly_song = get_billboard_chart(date)
    songs = pd.concat([songs, weekly_song])
    time.sleep(5)

# [전처리] ---------------------------------------------------------------
# week 데이터 처리
songs['Week'] = songs['Week'].str[8:]
songs.Week = songs.Week.str.split(' ')
songs['Year'] = songs.Week.str[2]
songs['Month'] = songs.Week.str[0]
songs['Day'] = songs.Week.str[1]
songs['Day'] = songs.Day.str.replace(',', '')

# month 매핑: 문자 -> 숫자
month = {'January': 1,
         'February': 2,
         'March': 3,
         'April': 4,
         'May': 5,
         'June': 6,
         'July': 7,
         'August': 8,
         'September': 9,
         'October': 10,
         'November': 11,
         'December': 12}
songs.Month = songs.Month.map(month)

def week_number(date_val):
    year = date_val.year
    month = date_val.month
    day = date_val.day
    # 일요일 시작
    cal = calendar.Calendar(calendar.SUNDAY)
    dates = cal.monthdatescalendar(year, month)
    for week_index, week in enumerate(dates):
        for day_index, date in enumerate(week):
            if date.month == month and date.day == day:
                return week_index + 1

songs.Week = songs.apply(lambda row: week_number(datetime.date(int(row['Year']), int(row['Month']), int(row['Day']))), axis = 1)
songs = songs[['Year', 'Month', 'Week', 'Rank', 'Title', 'Artist']]
songs.to_csv('billboardTop100.csv', index = False)


100%|██████████| 262/262 [27:24<00:00,  6.28s/it]


PermissionError: [Errno 13] Permission denied: 'billboardTop100.csv'